In [ ]:
!sudo python -m spacy download es_core_news_lg

In [ ]:
%load_ext autoreload
%autoreload 1
%aimport aymurai.pattern.fuzzymatch
%aimport aymurai.pattern.fuzzytagging

In [ ]:
import sys, os

path = '/resources/experiments/ia2-cli'
if not os.path.exists(path):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    !git clone https://github.com/instituciones-abiertas/ia2-cli /resources/experiments/ia2-cli 

sys.path.append('/resources/experiments/ia2-cli')

In [ ]:
from pipeline_components.entity_ruler import fetch_ruler_patterns_by_tag
from pipeline_components.entity_matcher import (
    ArticlesMatcher,
    EntityMatcher,
    ViolenceContextMatcher,
    matcher_patterns,
    fetch_cb_by_tag,
)
from pipeline_components.entity_custom import EntityCustom

In [ ]:
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
from aymurai.datasets.ar_juz_pcyf_10 import ArgentinaJuzgadoPCyF10Dataset
from aymurai.datasets.utils import load_documents_from
from aymurai.text.extraction import extract_document
import locale

locale.setlocale(locale.LC_ALL, "es_AR.UTF-8")


In [ ]:
public = ArgentinaJuzgadoPCyF10Dataset('latest')
private = ArgentinaJuzgadoPCyF10Dataset('private')
mexico = load_documents_from('/resources/data-mexico/')

In [ ]:
from spacy.pipeline import EntityRuler
from spacy.language import Language


@Language.factory(name="ia2_entity_ruler")
def ia2_entity_ruler(nlp, name):
    return EntityRuler(nlp)


@Language.factory(name="ia2_entity_matcher")
def ia2_entity_matcher(nlp, name):
    return EntityMatcher(
        nlp,
        matcher_patterns,
        after_callbacks=[cb(nlp) for cb in fetch_cb_by_tag("todas")],
    )

@Language.factory(name='ia2_entity_custom')
def ia2_entity_custom(nlp, name):
    return EntityCustom(nlp, "todas")


In [ ]:

nlp = spacy.load("es_core_news_lg")
nlp.add_pipe('ia2_entity_ruler')
nlp.add_pipe('ia2_entity_matcher')
nlp.add_pipe('ia2_entity_custom')


In [ ]:
register = public[400]
register

# anonimizados

In [ ]:
text = extract_document(register['path'], extension='odt')
doc = nlp(text)
displacy.render(doc, 'ent')

# mexico

In [ ]:
register = mexico[2]
doc = nlp(register['text'])
displacy.render(doc, 'ent')